# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!


  1. Use RAGAS to Generate Synthetic Data
  2. Load them into a LangSmith Dataset
  3. Evaluate our RAG chain against the synthetic test data
  4. Make changes to our pipeline
  5. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/jela.yap/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jela.yap/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/60 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 219)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 219)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

1. SingleHopSpecificQuerySynthesizer (50% distribution) creates straightforward questions that require information from a single document or node, like "What is the penalty for AI misuse?"

2. MultiHopAbstractQuerySynthesizer (25% distribution) generates complex questions requiring reasoning across multiple documents with abstract concepts, such as "How do different AI regulations compare in their approach to privacy protection?"

3. MultiHopSpecificQuerySynthesizer (25% distribution) creates questions that need specific information from multiple sources, like "What are the specific penalties mentioned in different sections of the AI bill?"

</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is CD0 Magazine about?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,CD0 Magazine is referenced in the context as a...,single_hop_specifc_query_synthesizer
1,Considering the opportunities and risks associ...,[AI presents enormous opportunities for the Ph...,The bill seeks to strike a careful balance bet...,single_hop_specifc_query_synthesizer
2,Who is PIA S. CAYETANO in relation to the AI r...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,PIA S. CAYETANO is the senator who introduced ...,single_hop_specifc_query_synthesizer
3,How Filipinos are affected by AI in the contex...,"[1 \na) Promote innovation, technological adva...",The context emphasizes the importance of prote...,single_hop_specifc_query_synthesizer
4,What is the ASI thing that is like super smart...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial Superintelligence (ASI) refers to h...,single_hop_specifc_query_synthesizer
5,How does the NAIC's plan to create an AI Ethic...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,The NAIC's plan to create an AI Ethics Review ...,multi_hop_abstract_query_synthesizer
6,How does the AI regulation act in the philippi...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The AI regulation act in the philippines empha...,multi_hop_abstract_query_synthesizer
7,How does the Philippine AI Regulation Act prom...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The Philippine AI Regulation Act emphasizes th...,multi_hop_abstract_query_synthesizer
8,WHy is the SENATE invlved in AI regualtion in ...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The SENATE is involved in AI regulation in the...,multi_hop_specific_query_synthesizer
9,How does the potential rise of ASI relate to t...,[<1-hop>\n\nAI presents enormous opportunities...,The context highlights that AI presents opport...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/60 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of the REPUBLIC OF TH...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The context discusses the TWENTIETH CONGRESS O...,single_hop_specifc_query_synthesizer
1,Whaat is the ris of ASI in AI polcy and how do...,[AI presents enormous opportunities for the Ph...,The context mentions that the global scientifi...,single_hop_specifc_query_synthesizer
2,What is the AI regulation act (AIRA)?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The Artificial Intelligence Regulation Act (AI...,single_hop_specifc_query_synthesizer
3,How does the policy address the development of...,"[1 \na) Promote innovation, technological adva...",The policy states that the Act shall regulate ...,single_hop_specifc_query_synthesizer
4,How does the Philippines' AI policy promote AI...,"[<1-hop>\n\n1 \na) Promote innovation, technol...",The Philippines' AI policy promotes the respon...,multi_hop_abstract_query_synthesizer
5,How can AI policies in the Philippines ensure ...,"[<1-hop>\n\n1 \na) Promote innovation, technol...",The policies aim to promote responsible AI use...,multi_hop_abstract_query_synthesizer
6,How does the risk-based regulatory framework f...,[<1-hop>\n\n1 \ndevelopment priorities. These ...,"The risk-based regulatory framework for AI, im...",multi_hop_abstract_query_synthesizer
7,How does the philippine ai law promote global ...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The philippine ai law promotes global ai gover...,multi_hop_abstract_query_synthesizer
8,How does the regulation of Artificiai Inteiiig...,"[<1-hop>\n\n1 \na) Promote innovation, technol...",The regulation of AI in the Philippines aims t...,multi_hop_specific_query_synthesizer
9,Considering the potential rise of Artificial S...,[<1-hop>\n\nAI presents enormous opportunities...,The Philippines should develop a comprehensive...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is a fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment of three (3) years to ten (10) years, or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

- `qa_evaluator`: Evaluates the basic question-answering quality by comparing the generated response against the expected answer, measuring factual accuracy and correctness.
- `labeled_helpfulness_evaluator`: Assesses whether the response is genuinely helpful to the user by comparing it against the reference answer, considering both correctness and utility of the information provided.

</div>

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'earnest-lunch-27' at:
https://smith.langchain.com/o/12018d00-fc5d-4253-9ba7-45146ba35e35/datasets/44f88d13-6fc4-4594-a4e9-97f96cfe8517/compare?selectedSessions=fdf05e54-938f-4fbb-9ee2-e6c843b87ab1




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the NAIC coordinate with the National...,The NAIC coordinates with the National Privacy...,None,The NAIC coordinates with the National Privacy...,1,1,0,1.775520,a36ebe01-b4c2-4e91-9a9c-52bf337013fd,4c7c85f4-82b8-4240-ae69-896dacc03281
1,How does the regulation of ASI relate to futur...,"Based on the provided context, the regulation ...",None,The regulation of ASI (Artificial Superintelli...,1,1,0,4.173344,3e43f6c3-570a-440b-8bc6-f27aeec8fe2a,ba0e20c9-ba08-4b0e-9410-9faa1803d057
2,Considering the potential rise of Artificial S...,I don't know.,None,The Philippines should develop a comprehensive...,0,0,0,1.821489,05bba6f6-7188-431e-a235-c3d93716c841,51a83ad5-4443-486a-a36e-3114ad9d4167
3,How does the regulation of Artificiai Inteiiig...,The regulation of Artificial Intelligence (AI)...,None,The regulation of AI in the Philippines aims t...,1,1,0,6.198162,53ebac6d-9148-4fd0-8ea9-e0fc790a4ed3,93aa9780-88e2-44b0-a099-cbc67bee68f3
4,How does the philippine ai law promote global ...,The Philippine AI law promotes global AI gover...,None,The philippine ai law promotes global ai gover...,1,0,0,8.373466,6a01b4ee-f753-4420-bdbf-2aa5914add29,e518766e-20ee-4bc7-9664-f06ee358d9b5
5,How does the risk-based regulatory framework f...,I don't know.,None,"The risk-based regulatory framework for AI, im...",0,0,0,1.155607,d4d61b87-a866-4763-9866-961abca021e7,24e28bed-09d5-48a2-b271-62c0e3f2d3e8
6,How can AI policies in the Philippines ensure ...,"Based on the provided context, AI policies in ...",None,The policies aim to promote responsible AI use...,1,1,0,5.753352,2dd34c76-087b-4dcb-874a-30cda041030f,50bf7c86-8392-4427-8cff-271a62f7c9ea
7,How does the Philippines' AI policy promote AI...,"Based on the provided context, the Philippines...",None,The Philippines' AI policy promotes the respon...,1,1,0,6.144013,01afc5fc-39cb-4179-bf9c-f4b08fdbb1c7,f0409e52-a232-4dd9-b6f8-5cdae4186171
8,How does the policy address the development of...,The policy regulates the development and use o...,None,The policy states that the Act shall regulate ...,1,0,0,3.072765,4884f7e3-57c5-4c82-9d38-a456f8511e3e,8120ce12-7759-4b86-a235-decf7bfa7cdb
9,What is the AI regulation act (AIRA)?,The Artificial Intelligence Regulation Act (AI...,None,The Artificial Intelligence Regulation Act (AI...,1,1,0,4.141724,882322b2-b9dc-47ce-8dda-1319f89f2fdb,7c94eef8-bbd8-4fe9-999b-f2f21dfd696f


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

Larger chunks (1000 vs 500 characters) provide more comprehensive context per retrieval, allowing the model to access broader information that might span across what would otherwise be multiple smaller chunks. This reduces the risk of fragmenting important information and helps maintain coherent context for complex questions. However, larger chunks may also introduce more irrelevant information and could exceed token limits, while smaller chunks might miss connections between related concepts but offer more precise retrieval.

</div>

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

Upgrading from text-embedding-3-small to text-embedding-3-large improves semantic understanding and retrieval accuracy. The larger model has higher dimensional representations and better captures nuanced relationships between concepts, leading to more relevant document retrieval. This means when users ask questions about AI bills, the system can better identify contextually relevant sections even when the exact keywords don't match, improving the overall quality of retrieved context and subsequent answers.

</div>

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

"Thank you for your thoughtful question about the importance of the Philippines AI Bill. Based on the context provided, this bill is truly significant because it aims to carefully balance encouraging technological innovation with ensuring that AI systems remain safe, ethical, transparent, and under meaningful human oversight. \n\nAs AI continues to transform various industries and aspects of society, the bill provides a national framework to guide its responsible and lawful development, keeping in mind the welfare and rights of every Filipino citizen. It reflects a hopeful and responsible vision where AI supports Filipino ingenuity and helps address national development challenges without causing harm or unintended consequences.\n\nThe bill also expresses the State's responsibility to prevent AI from being used to perpetrate crimes or abuse rights, showing consideration for both progress and protection. This kind of thoughtful legislation can help build trust and ensure that technology

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'flowery-scent-90' at:
https://smith.langchain.com/o/12018d00-fc5d-4253-9ba7-45146ba35e35/datasets/44f88d13-6fc4-4594-a4e9-97f96cfe8517/compare?selectedSessions=43f93da6-2aa5-43b0-ba1d-d73aa345d30c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the NAIC coordinate with the National...,Thank you for your thoughtful question about h...,None,The NAIC coordinates with the National Privacy...,1,1,1,4.562946,a36ebe01-b4c2-4e91-9a9c-52bf337013fd,2c308d72-4953-4347-9fa5-387bb78d3339
1,How does the regulation of ASI relate to futur...,Thank you for your thoughtful question. Based ...,None,The regulation of ASI (Artificial Superintelli...,1,1,1,3.784292,3e43f6c3-570a-440b-8bc6-f27aeec8fe2a,7e6cd22d-2f13-425f-b206-615c451345d9
2,Considering the potential rise of Artificial S...,Thank you for your thoughtful question. It’s c...,None,The Philippines should develop a comprehensive...,1,1,1,11.111130,05bba6f6-7188-431e-a235-c3d93716c841,90f48f1c-0680-4163-a5f7-b2af52095ffd
3,How does the regulation of Artificiai Inteiiig...,Thank you for your thoughtful question about h...,None,The regulation of AI in the Philippines aims t...,1,1,1,8.156418,53ebac6d-9148-4fd0-8ea9-e0fc790a4ed3,e1a05ec8-da08-45ba-a969-51ac69d5694c
4,How does the philippine ai law promote global ...,Thank you for your thoughtful question. From t...,None,The philippine ai law promotes global ai gover...,0,0,1,6.119456,6a01b4ee-f753-4420-bdbf-2aa5914add29,30d91b6c-e434-48b6-8608-ce542b7d0e96
5,How does the risk-based regulatory framework f...,Thank you for your thoughtful question. From t...,None,"The risk-based regulatory framework for AI, im...",1,0,1,7.970902,d4d61b87-a866-4763-9866-961abca021e7,e79d1dec-20a2-47c2-a8c6-0acfe33d9fb2
6,How can AI policies in the Philippines ensure ...,Thank you for your thoughtful question. It’s c...,None,The policies aim to promote responsible AI use...,1,1,1,5.118604,2dd34c76-087b-4dcb-874a-30cda041030f,ebbb62c7-2142-461a-8734-ae6276d0760c
7,How does the Philippines' AI policy promote AI...,Thank you for your thoughtful question. Based ...,None,The Philippines' AI policy promotes the respon...,1,1,1,5.940599,01afc5fc-39cb-4179-bf9c-f4b08fdbb1c7,e3a1032c-fd99-4e3b-94cf-4e38cac61ea9
8,How does the policy address the development of...,Thank you for your thoughtful question. Based ...,None,The policy states that the Act shall regulate ...,1,1,1,3.237480,4884f7e3-57c5-4c82-9d38-a456f8511e3e,595e3c99-d243-46b9-84fc-8d556aebd521
9,What is the AI regulation act (AIRA)?,Thank you for your thoughtful question. Based ...,None,The Artificial Intelligence Regulation Act (AI...,1,1,1,3.916697,882322b2-b9dc-47ce-8dda-1319f89f2fdb,bf55ae89-895e-41e3-8175-35626c8c207f


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

![LangSmith Comparison](images/chains.jpeg)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

The empathy-enhanced chain showed improved empathy scores due to the explicit empathy instruction in the prompt. Helpfulness scores slightly increased because the empathetic approach often leads to more comprehensive, user-focused responses. Correctness also slightly improved since the larger chunks and better embeddings provide richer context, though the empathy focus might occasionally reduce precision. This shows how the combination of larger chunks, better embeddings, and empathy-focused prompting can create a more user-centric system that balances accuracy with emotional intelligence, though this might come at the cost of some directness in factual responses.

</div>